In [4]:
!pip install flask

from flask import Flask, render_template, request, redirect, url_for
import os
import numpy as np
from PIL import Image
import json
import requests

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # Max 16MB upload

# Load ImageNet class index
def load_class_index():
    with open('imagenet_class_index.json') as f:
        return json.load(f)

# Function to preprocess the uploaded image
def preprocess_image(image_path):
    # Load the image
    image = Image.open(image_path)
    # Resize the image to 224x224 (the input size expected by MobileNetV2)
    image = image.resize((224, 224))
    # Convert the image to a numpy array
    image_array = np.array(image)
    # Normalize the image array to [0, 1]
    image_array = image_array / 255.0
    # Add a batch dimension
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

# Function to make an inference request to OpenVINO endpoint
def predict(image_array, inference_url):
    # Convert the image array to a list of floats
    image_list = image_array.astype(np.float32).tolist()
    
    # Prepare the payload
    payload = {
        "inputs": [
            {
                "name": "input_1",
                "shape": [1, 224, 224, 3],
                "datatype": "FP32",
                "data": image_list
            }
        ]
    }
    
    # Prepare the headers
    headers = {'Content-Type': 'application/json'}
    
    # Make a POST request to the inference URL
    response = requests.post(inference_url, headers=headers, json=payload)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        predictions = response.json()
        return predictions
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.text)
        return None

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Check if a file was uploaded
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        
        # Save the uploaded file
        file_path = file.filename
        file.save(file_path)

        # Preprocess the image
        image_array = preprocess_image(file_path)
        
        # OpenVINO inference URL
        inference_url = '<inference-endpoint>'

        # Get predictions
        output_prediction = predict(image_array, inference_url)
        
        if output_prediction:
            # Load class index
            class_idx = load_class_index()
            
            # Extract prediction data
            predictions = output_prediction['outputs'][0]['data']
            predictions_np = np.array(predictions)
            
            # Get the predicted class
            max_index = np.argmax(predictions_np)
            label = class_idx[str(max_index)][1]
            
            # Return the result to the index page
            return render_template('index.html', label=label, image_url=url_for('static', filename=f'uploads/{file.filename}'))
        else:
            return render_template('index.html', label="Prediction failed.", image_url=None)
    
    return render_template('index.html', label=None, image_url=None)

if __name__ == "__main__":
    app.run(debug=False, port=5000)


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Sep/2024 07:03:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2024 07:03:47] "POST / HTTP/1.1" 200 -
